In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
import statsmodels.graphics.tsaplots as sgt
from statsmodels.tsa.stattools import adfuller


In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
df=pd.read_csv('Price.csv')
df_comp=df.copy()

df.date=pd.to_datetime(df.date,dayfirst=True)
df.set_index('date',inplace=True)

df=df.resample('B').mean()

In [ ]:
df

,spx,dax,ftse,nikkei
date,,,,
1994-01-07,469.900000,2224.95,3445.980000,18124.01
1994-01-10,475.270000,2225.00,3440.580000,18443.44
1994-01-11,474.130000,2228.10,3413.770000,18485.25
1994-01-12,474.170000,2182.06,3372.020000,18793.88
1994-01-13,472.470000,2142.37,3360.010000,18577.26
...,...,...,...,...
2018-01-23,2839.130362,13559.60,7731.827774,24124.15
2018-01-24,2837.544008,13414.74,7643.428966,23940.78
2018-01-25,2839.253031,13298.36,7615.839954,23669.49


In [ ]:
df=df.fillna(method='bfill')

GARCH

In [ ]:
import statsmodels.tsa.stattools as sts

In [ ]:
def difference(dataset,interval=1):
  diff=list()
  for i in range(interval,len(dataset)):
    value=dataset[i]-dataset[i-interval]
    diff.append(value)
  return diff

In [ ]:
ftse_diff=difference(df.ftse)

In [ ]:
df=df.iloc[1:,:]

In [ ]:


df['ftse_diff']=ftse_diff

In [ ]:
pip install arch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.0/908.0 KB 11.7 MB/s eta 0:00:00


In [ ]:
from arch import arch_model

In [ ]:
model_garch_21=arch_model(df.ftse_diff,mean='Constant',vol='GARCH',p=2,q=1)

result_garch_21= model_garch_21.fit(update_freq=5)

result_garch_21.summary()

Iteration:      5,   Func. Count:     38,   Neg. LLF: 34785.81688022874
Iteration:     10,   Func. Count:     71,   Neg. LLF: 33304.33695300504
Iteration:     15,   Func. Count:    104,   Neg. LLF: 33293.84105272149
Optimization terminated successfully    (Exit mode 0)
            Current function value: 33293.84105225775
            Iterations: 16
            Function evaluations: 111
            Gradient evaluations: 16


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Constant Mean - GARCH Model Results                      
==============================================================================
Dep. Variable:              ftse_diff   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -33293.8
Distribution:                  Normal   AIC:                           66597.7
Method:            Maximum Likelihood   BIC:                           66631.4
                                        No. Observations:                 6276
Date:                Thu, Mar 09 2023   Df Residuals:                     6275
Time:                        15:03:25   Df Model:                            1
                               Mean Model                               
========================================================================
                 coef    std err          t      P>|t|  95.0% Conf. Int.
------------------------------------------------------------------------
mu             1.7276      0.504      3.428  6.075e-04 [  0.740,  2.715]
                               Volatility Model                              
=============================================================================
                 coef    std err          t      P>|t|       95.0% Conf. Int.
-----------------------------------------------------------------------------
omega          9.2082      3.549      2.595  9.463e-03      [  2.253, 16.163]
alpha[1]       0.0672  1.661e-02      4.044  5.246e-05  [3.463e-02,9.975e-02]
alpha[2]   5.1789e-19  2.002e-02  2.587e-17      1.000 [-3.924e-02,3.924e-02]
beta[1]        0.9325  1.041e-02     89.595      0.000      [  0.912,  0.953]
=============================================================================

Covariance estimator: robust
"""

In [ ]:
forcast_garch =result_garch_21.forecast(horizon=1)

In [ ]:
pre_et = np.sqrt(forcast_garch.variance.values[-1,:])

In [ ]:
pre_et

array([37.21780962])

In [ ]:
mu = forcast_garch.mean.values[-1,:]

In [ ]:
mu

array([1.72756465])

In [ ]:
df

,spx,dax,ftse,nikkei,ftse_diff
date,,,,,
1994-01-10,475.270000,2225.00,3440.580000,18443.44,-5.400000
1994-01-11,474.130000,2228.10,3413.770000,18485.25,-26.810000
1994-01-12,474.170000,2182.06,3372.020000,18793.88,-41.750000
1994-01-13,472.470000,2142.37,3360.010000,18577.26,-12.010000
1994-01-14,474.910000,2151.05,3400.560000,18973.70,40.550000
...,...,...,...,...,...
2018-01-23,2839.130362,13559.60,7731.827774,24124.15,16.386240
2018-01-24,2837.544008,13414.74,7643.428966,23940.78,-88.398808
2018-01-25,2839.253031,13298.36,7615.839954,23669.49,-27.589012


In [ ]:
#train test split


size= int(len(df)*0.8)
train_data=df.iloc[:size,:]
test_data=df.iloc[size:,:]

In [ ]:
model_arima=sm.tsa.arima.ARIMA(train_data.ftse,order=(3,1,2))

result_arima=model_arima.fit()

result_arima.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                   ftse   No. Observations:                 5020
Model:                 ARIMA(3, 1, 2)   Log Likelihood              -27587.161
Date:                Thu, 09 Mar 2023   AIC                          55186.323
Time:                        15:09:18   BIC                          55225.448
Sample:                    01-10-1994   HQIC                         55200.033
                         - 04-05-2013                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1         -0.5674      0.080     -7.130      0.000      -0.723      -0.411
ar.L2         -0.4416      0.088     -5.030      0.000      -0.614      -0.270
ar.L3         -0.0938      0.010     -9.540      0.000      -0.113      -0.075
ma.L1          0.5491      0.080      6.860      0.000       0.392       0.706
ma.L2          0.3854      0.088      4.362      0.000       0.212       0.559
sigma2      3484.5738     42.019     82.929      0.000    3402.218    3566.929
===================================================================================
Ljung-Box (L1) (Q):                   0.00   Jarque-Bera (JB):              2995.39
Prob(Q):                              0.96   Prob(JB):                         0.00
Heteroskedasticity (H):               1.89   Skew:                            -0.26
Prob(H) (two-sided):                  0.00   Kurtosis:                         6.75
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [ ]:
pred = result_arima.forecast()[0]

next_price = pred + mu + pre_et

In [ ]:
next_price

array([6302.53825426])